## 测试 rust 版本 czsc 库的使用

----

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
from rs_czsc import RawBar, Freq, CZSC, Direction, Mark, format_standard_kline

def create_test_bars(count=1000):
    """创建测试用的K线数据"""
    # from czsc import mock

    # df = mock.generate_klines()
    df = pd.read_feather(r"A:\桌面临时数据\行情数据\BTCUSDT5分钟行情.feather")
    df = df.reset_index(drop=True)
    bars = format_standard_kline(df.tail(count))
    return bars


### 一、性能测试
---

### 二、功能测试
---

#### 测试 Signal

In [1]:
from rs_czsc import Signal

s1 = Signal(key="30分钟_D1_表里关系V230101", value="向上_任意_任意_0")
print(s1)

s2 = Signal(k1="60分钟", k2="D1", k3="表里关系V230101", v1="向上", v2="任意", v3="任意", score=0)
print(s2)

s3 = Signal("30分钟_D1_表里关系V230101_向上_任意_任意_0")
print(s3)

Signal('30分钟_D1_表里关系V230101_向上_任意_任意_0')
Signal('60分钟_D1_表里关系V230101_向上_任意_任意_0')
Signal('30分钟_D1_表里关系V230101_向上_任意_任意_0')


In [2]:
assert str(s1) == "30分钟_D1_表里关系V230101_向上_任意_任意_0"

AssertionError: 

#### 测试 Event 对象

In [6]:
# python 版本：https://github.com/waditu/czsc/blob/0.9.68/czsc/objects.py#L630
from rs_czsc import Event, Signal, Operate

event = Event(name='平多', operate=Operate.LE, 
              signals_all=[Signal("日线_D0停顿分型_BE辅助V230106_看空_强_任意_0")])

e1 = event.dump()
print(e1, "\n\n")
e2 = Event.load(e1)
print(e2)

{'name': '平多#A547', 'operate': 'LE', 'signals_all': ['日线_D0停顿分型_BE辅助V230106_看空_强_任意_0'], 'signals_any': [], 'signals_not': []} 


Event[平多#A547]: LE (all:1, any:0, not:0)


AttributeError: 'builtins.Event' object has no attribute 'opens'

In [ ]:
from rs_czsc import Operate

op = Operate.LE
assert op.value == "平多"

In [15]:
from rs_czsc import Event, Signal, Operate, Position


base_freq = '30分钟'

opens = [
    {
        "operate": "开多",
        "signals_all": [f"{base_freq}_D1_表里关系V230101_向上_任意_任意_0"],
        "signals_any": [],
        "signals_not": [f"{base_freq}_D1_涨跌停V230331_涨停_任意_任意_0"]
    },
]

exits = []

pos = Position(
    name=f"{base_freq}_LONG_E001",
    symbol="xxx",
    opens=[Event.load(x) for x in opens],
    exits=[Event.load(x) for x in exits],
    interval=3600 * 4,
    timeout=16,
    stop_loss=300,
)

p1 = pos.dump()
print(p1, "\n\n")
p2 = Position.load(p1)
print(p2)
print(p2.dump())

{'symbol': 'xxx', 'name': '30分钟_LONG_E001', 'opens': [{'name': 'LO#6B06', 'operate': 'LO', 'signals_all': [{'key': '30分钟_D1_表里关系V230101', 'value': '向上_任意_任意_0'}], 'signals_any': [], 'signals_not': [{'key': '30分钟_D1_涨跌停V230331', 'value': '涨停_任意_任意_0'}]}], 'exits': [], 'interval': 14400, 'timeout': 16, 'stop_loss': 300.0, 'T0': False, 'pairs': [], 'holds': []} 


Position(name='30分钟_LONG_E001', symbol='xxx', opens=1, exits=0)
{'symbol': 'xxx', 'name': '30分钟_LONG_E001', 'opens': [{'name': 'LO#6B06', 'operate': 'LO', 'signals_all': [{'key': '30分钟_D1_表里关系V230101', 'value': '向上_任意_任意_0'}], 'signals_any': [], 'signals_not': []}], 'exits': [], 'interval': 14400, 'timeout': 16, 'stop_loss': 300.0, 'T0': False, 'pairs': [], 'holds': []}


In [ ]:
import hashlib
open_signals_all = [
    f"{base_freq}_D1DIF10斜率T80_BS辅助V231019_看多_任意_任意_0"
]
open_signals_all = [Signal(s) for s in open_signals_all]
open_signals_all = [{"key": s.key, "value": s.value} for s in open_signals_all]
pos_dict = pos.dump()
sig_hash = hashlib.md5(f"{open_signals_all}".encode('utf-8')).hexdigest()[:8].upper()
pos_dict['name'] = f"{pos.name}#{sig_hash}"
pos_dict['opens'][0]['signals_all'].extend(open_signals_all)

In [23]:
pos_dict
Position.load(pos_dict)

TypeError: 'str' object cannot be converted to 'PyDict'